In [1]:
import pandas as pd
import numpy as np

from scripts.pre_process_dataset import PreProcess
from scripts.import_dataset import ImportDataset
from scripts.plots import PlotReport

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
# mpl.rcParams["text.usetex"] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 16

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)

[nltk_data] Downloading package wordnet to /home/dip07/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/dip07/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-12-04 10:19:22.873693: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 10:19:22.914451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 10:19:24.048860: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical r

In [2]:
data = ImportDataset()
df = data.read_dataset()
print(f"✅ Combined dataset size: {df.shape[0]} samples")

✅ Combined dataset size: 14438 samples


In [3]:
clean_df = PreProcess(df)
df = clean_df.preprocess_dataset()
df.head(1)

,condition_label,medical_abstract
0,0,tissue change around loose prosthesis canine m...


In [4]:
X = df['medical_abstract']
y = df['condition_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_test.shape[0]}")

Training set size: 10106
Validation set size: 4332


In [5]:
# Settings
MAX_WORDS = 5000
MAX_LEN = 200

# Tokenize
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')


# Class weights
class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights_array[i] for i in range(5)}
print(f"Vocab: {MAX_WORDS}, Max Length: {MAX_LEN}")
print(f"Train shape: {X_train_pad.shape}")

Vocab: 5000, Max Length: 200
Train shape: (10106, 200)


# Build, Train, and Evaluate LSTM Model


In [ ]:
# Build LSTM
model_lstm = Sequential([
    Embedding(MAX_WORDS, 64, input_length=MAX_LEN),
    LSTM(32, return_sequences=True, dropout=0.2),
    GlobalMaxPooling1D(),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

model_lstm.compile(
    optimizer=keras.optimizers.Adam(0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_lstm = model_lstm.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    verbose=1
)

# Evaluate
# y_pred_lstm = np.argmax(model_lstm.predict(X_test_pad, verbose=0), axis=1)
# acc_lstm = accuracy_score(y_test, y_pred_lstm)
# print(f"\nLSTM Accuracy: {acc_lstm:.4f} ({acc_lstm*100:.2f}%)")


Epoch 1/20


2025-12-04 10:19:35.628773: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


127/127 ━━━━━━━━━━━━━━━━━━━━ 16s 113ms/step - accuracy: 0.2626 - loss: 1.7549 - val_accuracy: 0.3877 - val_loss: 1.6803
Epoch 2/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.4041 - loss: 1.5223 - val_accuracy: 0.5114 - val_loss: 1.3866
Epoch 3/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.5000 - loss: 1.3504 - val_accuracy: 0.5262 - val_loss: 1.2833
Epoch 4/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 109ms/step - accuracy: 0.5280 - loss: 1.2629 - val_accuracy: 0.5232 - val_loss: 1.2356
Epoch 5/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 107ms/step - accuracy: 0.5403 - loss: 1.2129 - val_accuracy: 0.5237 - val_loss: 1.2180
Epoch 6/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 113ms/step - accuracy: 0.5501 - loss: 1.1746 - val_accuracy: 0.5203 - val_loss: 1.1931
Epoch 7/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.5727 - loss: 1.1233 - val_accuracy: 0.5213 - val_loss: 1.2061
Epoch 8/20
 46/127 ━━━━━━━━━━━━━━━━━━━━ 8s 100ms/step - accuracy: 0.5814 - loss: 1.0769

In [ ]:
loss, acc = model_lstm.evaluate(X_test_pad, y_test)
print("Accuracy:", acc)

# Classification Reports of the RNN Models

In [ ]:
# Classification report for LSTM
# print("\n LSTM Classification Report ")
# print(classification_report(y_test, y_pred_lstm, target_names=[f'class_{i}' for i in range(5)]))

plot = PlotReport(model_lstm, X_test_pad, y_test)
plot.get_confusion_matrix(True)
plot.print_confusion_matrix("Confusion_RNN.pdf")

# Visualize Model Performance and Training Results


In [ ]:
# Graph 1: Model Comparison
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(history_lstm.history['accuracy'], label='Train', linewidth=2)
ax.plot(history_lstm.history['val_accuracy'], label='Validation', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('LSTM Training History', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)


plt.suptitle('RNN Models Performance Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
df_labels = pd.read_csv("data/medical_tc_labels.csv")
df_labels.loc[df_labels['condition_label'] == 5, 'condition_label'] = 0
df_labels['name'] = df_labels['condition_name'].str.split(n=1, expand=True)[0]
df_labels = df_labels.sort_values(by='condition_label')
df_labels